In [1]:
import os
import numpy as np
import nglview as nv
from pymatgen.core import Structure, Molecule
from pymatgen.io.ase import AseAtomsAdaptor
from ase.io import read, write

from mkite_catalysis.runners.coverage import CoverageGenerator
from mkite_catalysis.runners.deduplicate import Deduplicator

## Loading the basics

In [2]:
helium = Molecule(species=["He"], coords=[[0, 0, 0]])

In [3]:
facet_dir = "../data/01-facets"
facets = [100, 111, 211, 331, 410, 711]

cutoffs = {
    111: 1.0,
    100: 1.0,
    211: 1.4,
    331: 1.6,
    410: 1.2,
    711: 1.8
}

surfaces = {}
for facet in facets:
    fname = os.path.join(facet_dir, f"{facet}.POSCAR")
    surfaces[facet] = Structure.from_file(fname)

## Making the sites

In [4]:
for facet, surf in surfaces.items():
    covgen = CoverageGenerator(
        surf,
        helium,
        surface_height=cutoffs[facet],
        adsorption_height=2,
        distance_threshold=2,
    )

    sites = covgen.get_sites()

    new = surf.copy()

    for xyz in sites:
        new.append(
            species="He",
            coords=xyz,
            coords_are_cartesian=True,
            properties={"selective_dynamics": [False, False, False]}
        )
    
    new.to(f"../data/01-facets-sites/{facet}.cif")